# Submit Nanoribbon Workchain

In [ ]:
%aiida

In [ ]:
# General imports
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from ase.data import vdw_radii
from IPython.display import display, clear_output, HTML
import itertools

# AiiDA & AiiDA lab imports
from aiida.engine import calcfunction
from aiida.orm import SinglefileData
from aiidalab_widgets_base.utils import string_range_to_set, set_to_string_range

from nanoribbon.viewers import Smiles2GNRWidget
from aiidalab_widgets_base import CodeDropdown, StructureManagerWidget, StructureBrowserWidget, StructureUploadWidget, SubmitButtonWidget, SmilesWidget
from aiida.engine import submit

# Work Chains
NanoribbonWorkChain = WorkflowFactory('nanoribbon')

In [ ]:
structure_selector = StructureManagerWidget(importers=[
    ("SMILES2GNR", Smiles2GNRWidget()),
    ("Import from computer", StructureUploadWidget()),
    ("AiiDA database", StructureBrowserWidget()),
    ],
    storable=False,node_class='StructureData')
display(structure_selector)

In [ ]:
style = {'description_width': '120px'}
layout = {'width': '70%'}

pw_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pw')
pp_code_dropdown = CodeDropdown(input_plugin='quantumespresso.pp')
projwfc_code_dropdown = CodeDropdown(input_plugin='quantumespresso.projwfc')

text_calc_description = ipw.Text(description='Calculation Name: ',
                          placeholder='A great name.',
                          style=style, layout=layout)
spin_d = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin DOWN',
                                    style=style, layout={'width': '60%'})
spin_u = ipw.Text(placeholder='1..10 15',
                                    description='IDs atoms spin DOWN',
                                    style=style, layout={'width': '60%'})
display(ipw.VBox([pw_code_dropdown,pp_code_dropdown,projwfc_code_dropdown, text_calc_description]))

In [ ]:
## spin of structure in strcture browser
def check_spins(atoms):
    su=np.where(tags == 1)[0].tolist()
    sd=np.where(tags == 2)[0].tolist()
    su_str=set_to_string_range(su)
    sd_str=set_to_string_range(sd)
    #spin_d.value=sd_str
    #spin_u.value=su_str
    return su_str,sd_str


@calcfunction
def set_spins(atoms,su,sd,description):
        
    tags = np.zeros(len(atoms))
    for u in  string_range_to_sets(su):
        tags[u]=1
    for d in  string_range_to_sets(sd):
        tags[d]=2    

    nu = len(string_range_to_sets(su))
    nd = len(string_range_to_sets(sd))
    if nu == 0 and nd  == 0:
        add_name='no_spin'
    else:
        add_name = abs(nu - nd)
        
    atoms.set_tags(tags)

    new_struct = StructureData(ase=atoms)
    
    # ensure that tags got correctly translated into kinds 
    for t1, k in zip(tags, new_struct.get_site_kindnames()):
        t2 = int(k[-1]) if k[-1].isnumeric() else 0
        assert t1==t2
    
    new_struct.store()
    print("Stored in AiiDA: "+repr(s))        
        
    return new_struct

In [ ]:
def on_submit():    
    ## CHECK spin consistency and create new structure in case mismatch
    atoms = structure_selector.structure #ASE atoms
    description = structure_selector.structure_node.descritption
    ## spin from strucure browser
    su_str, sd_str = check_spins(atoms)
    ## spin in input widgets
    input_su = set_to_string_range(string_range_to_sets(pin_u.value))
    input_sd = set_to_string_range(string_range_to_set(spin_d.value))
    spin_mismatch = (input_su != su_str and input_su != sd_str) or (input_sd != su_str and input_sd != sd_str)
    if spin_mismatch :
        node_to_submit = set_spins(atoms,input_su,input_sd,description)
    else:
        node_to_submit = structure_selector.structure_node
        
    with submit_out:
        clear_output()
        if not structure_selector.structure_node:
            print("Please select a structure.")
            return
        if not pw_code_dropdown.selected_code:
            print("Please select a pw code.")
            return
        if not pp_code_dropdown.selected_code:
            print("Please select a pp code.")
            return
        if not projwfc_code_dropdown.selected_code:
            print("Please select a projwfc code.")
            return        
        if len(text_calc_description.value) < 3:
            print("Please enter a calculation name.")
            return

        #pw_code, pp_code, projwfc_code = qe_code_groups[drop_codes.value]
        builder = NanoribbonWorkChain.get_builder()
        builder.pw_code = pw_code_dropdown.selected_code
        builder.pp_code = pp_code_dropdown.selected_code
        builder.projwfc_code = projwfc_code_dropdown.selected_code
        builder.structure = node_to_submit
        builder.precision = Float(1.0) #Float(slider_precision.value)
        builder.pseudo_family = Str('TEST_PSEUDO') # Str('SSSP_precision_v1.0')
        builder.metadata = {
            "description": text_calc_description.value,
            "label": "NanoribbonWorkChain",
        }
        return builder

In [ ]:
btn_submit = SubmitButtonWidget(NanoribbonWorkChain,input_dictionary_function=on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)